# Importation des bibliothèques

In [40]:
import numpy as np
import pandas as pd


import re
import spacy
import fr_core_news_sm
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords


from sklearn.preprocessing import FunctionTransformer

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split

import sklearn.metrics

import matplotlib.pyplot as plt
import seaborn as sns

# Chargement des données

In [63]:
#DONNEE BRUTE

df = pd.read_excel("C:/Users/ET03349/Desktop/Sémantique/Lot_voix_ca/labels-voix-ca-lot-5.xlsx")
df = df[['text', 'category', 'is_project']]

In [64]:
df.category.unique()

array(['Date', 'Point_d_attention_risques',
       'Entree_en_relation_Conquetes', 'Montant',
       'Bilan_suivi_relationnel', '_pargne_disponible', 'Credit_conso',
       'Reclamation_Insatisfaction_client', '_pargne_terme',
       'Credit_vehicule', 'Assurance_de_Personnes', 'Credit_agri_pro',
       'Non_specifie_Credit_Assurance_Epargne_', 'Assurance_de_Biens',
       'Credit_habitat', 'BAQ_Services_Bancaires', '_venements_de_vie',
       'Projet_exceptionnel_', 'Credit_travaux'], dtype=object)

# Encoding dataframe

In [65]:
df_s = df.groupby(['text', 'category'])['is_project'].first().unstack(-1)
df_s = (~df_s.isna()).astype(int)

# Les noms des colonnes cibles
category_cols = ['Montant', 'Credit_vehicule', 'Date', 'Bilan_suivi_relationnel',
       '_pargne_terme', 'Credit_travaux', 'Entree_en_relation_Conquetes',
       'Point_d_attention_risques', 'Reclamation_Insatisfaction_client',
       'Credit_conso', '_venements_de_vie', '_pargne_disponible',
       'Non_specifie_Credit_Assurance_Epargne_', 'Assurance_de_Biens',
       'Assurance_de_Personnes', 'BAQ_Services_Bancaires',
       'Credit_habitat', 'Credit_agri_pro', 'Projet_exceptionnel_']

df_s = df_s.reset_index()

In [66]:
print("Number of rows in data =",df_s.shape[0])
print("Number of columns in data =",df_s.shape[1])
print("\n")
print("**Sample data:**")
df_s.head()

Number of rows in data = 1145
Number of columns in data = 20


**Sample data:**


category,text,Assurance_de_Biens,Assurance_de_Personnes,BAQ_Services_Bancaires,Bilan_suivi_relationnel,Credit_agri_pro,Credit_conso,Credit_habitat,Credit_travaux,Credit_vehicule,Date,Entree_en_relation_Conquetes,Montant,Non_specifie_Credit_Assurance_Epargne_,Point_d_attention_risques,Projet_exceptionnel_,Reclamation_Insatisfaction_client,_pargne_disponible,_pargne_terme,_venements_de_vie
0,\n madame Richard à déménagé. \n\n travaille L...,1,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0
1,\n madame deboise travaille pour médecin sans ...,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
2,\n vente immobilière en cours. Épargne prévu...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0
3,"\nsuite entretien 18 ans, il existe un ovp de ...",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1
4,\ntravaux 15 ou 20 000 € sur 5 ou 6 ans. Mensu...,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0


In [67]:
df_s.describe()

category,Assurance_de_Biens,Assurance_de_Personnes,BAQ_Services_Bancaires,Bilan_suivi_relationnel,Credit_agri_pro,Credit_conso,Credit_habitat,Credit_travaux,Credit_vehicule,Date,Entree_en_relation_Conquetes,Montant,Non_specifie_Credit_Assurance_Epargne_,Point_d_attention_risques,Projet_exceptionnel_,Reclamation_Insatisfaction_client,_pargne_disponible,_pargne_terme,_venements_de_vie
count,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000
mean,0.172052,0.177293,0.159825,0.179913,0.034061,0.075983,0.204367,0.030568,0.051528,0.640175,0.225328,0.409607,0.166812,0.257642,0.065502,0.070742,0.145852,0.328384,0.084716
std,0.377591,0.382083,0.366604,0.384283,0.181466,0.265086,0.403414,0.172218,0.221169,0.480159,0.417980,0.491976,0.372971,0.437527,0.247518,0.256506,0.353112,0.469831,0.278581
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Visualisation de donnée

# Suppresion collonne peut représenter

In [28]:
df_s = df_s.drop(['Credit_vehicule', 'Credit_travaux', 'Credit_agri_pro', '_venements_de_vie', 'Credit_conso', 'Reclamation_Insatisfaction_client'], axis=1)

# Prepross texte

In [68]:
df_s.text = df_s.text.astype(str)

In [69]:
transformer = FunctionTransformer()
transformer.transform(df_s['text'])

0       \n madame Richard à déménagé. \n\n travaille L...
1       \n madame deboise travaille pour médecin sans ...
2       \n vente immobilière en cours.   Épargne prévu...
3       \nsuite entretien 18 ans, il existe un ovp de ...
4       \ntravaux 15 ou 20 000 € sur 5 ou 6 ans. Mensu...
                              ...                        
1140    w chez Phyto control, situation compliquée dep...
1141    à réception de l'avis d'imposition faire l'ouv...
1142                 échange téléphonique sur les mesures
1143    épargne salariale et de proposer ce jour Monsi...
1144    étude de réaménagement ce jour        proposit...
Name: text, Length: 1145, dtype: object

In [70]:
# Chargement stopwords français
stop_words = stopwords.words('french')
    
#Initialize stemmer, which will take words and convert words to their "stem," e.g. Playing-> Play
ps = PorterStemmer() 

# Enlever les charactères nons alphabétiques, espaces et convertir toutes les lettres en minuscules
# References: https://www.analyticsvidhya.com/blog/2019/04/predicting-movie-genres-nlp-multi-label-classification/
def clean_text(txt): 
    txt= txt.lower()   #lowercase
    txt= re.sub("[^a-zA-Z]"," ",txt) #Remove everything except alphabetical characters 
    txt= word_tokenize(txt) #tokenize (split into list and remove whitespace)
    
    #initialize list to store clean text
    clean_text=""
      
    #iterate over each word
    for w in txt:      
        #remove stopwords
        if w not in stop_words:
            #stem=ps.stem(w) #stem 
            stem=w
            clean_text += stem +" " 
    return clean_text


text_new=[] #declare a list to hold new movies

for cell in df_s['text']:    
    txt= clean_text(cell)
    text_new.append(txt)
    
#add new info column to the dataframe
df_s['text'] = text_new 
df_s.head(1)

category,text,Assurance_de_Biens,Assurance_de_Personnes,BAQ_Services_Bancaires,Bilan_suivi_relationnel,Credit_agri_pro,Credit_conso,Credit_habitat,Credit_travaux,Credit_vehicule,Date,Entree_en_relation_Conquetes,Montant,Non_specifie_Credit_Assurance_Epargne_,Point_d_attention_risques,Projet_exceptionnel_,Reclamation_Insatisfaction_client,_pargne_disponible,_pargne_terme,_venements_de_vie
0,madame richard nag travaille mas azil gendmrie...,1,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0


In [71]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import sys
import warnings

data = df_s

if not sys.warnoptions:
    warnings.simplefilter("ignore")
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext

def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

data['text'] = data['text'].str.lower()
data['text'] = data['text'].apply(cleanHtml)
data['text'] = data['text'].apply(cleanPunc)
data['text'] = data['text'].apply(keepAlpha)

In [72]:
stop_words = set(stopwords.words('english'))
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)

def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)
data['text'] = data['text'].apply(removeStopWords)

In [73]:
stemmer = SnowballStemmer("french")

def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence
data['text'] = data['text'].apply(stemming)

# Modele

In [74]:
pipe = make_pipeline(
    CountVectorizer(),
    TfidfTransformer(),
    # Pour gérer les cas multi-label
    MultiOutputClassifier(
    # Le solveur par defaut est lent dans le cas sparse
       estimator=LogisticRegression(solver='liblinear')
    )
)

pipe.fit(data["text"], df_s[category_cols])

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('tfidftransformer', TfidfTransformer()),
                ('multioutputclassifier',
                 MultiOutputClassifier(estimator=LogisticRegression(solver='liblinear')))])

## train test

In [75]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], df_s[category_cols], random_state = 0)

In [76]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('tfidftransformer', TfidfTransformer()),
                ('multioutputclassifier',
                 MultiOutputClassifier(estimator=LogisticRegression(solver='liblinear')))])

# Prediction 

In [77]:
predict = pipe.predict(X_test)

In [78]:
from sklearn import metrics
print(metrics.classification_report(y_test, predict, target_names=category_cols))

                                        precision    recall  f1-score   support

                               Montant       0.90      0.58      0.71       126
                       Credit_vehicule       0.00      0.00      0.00        14
                                  Date       0.71      0.98      0.82       191
               Bilan_suivi_relationnel       0.00      0.00      0.00        49
                         _pargne_terme       0.95      0.40      0.56        92
                        Credit_travaux       0.00      0.00      0.00        12
          Entree_en_relation_Conquetes       1.00      0.16      0.28        67
             Point_d_attention_risques       1.00      0.14      0.24        65
     Reclamation_Insatisfaction_client       0.00      0.00      0.00        20
                          Credit_conso       0.00      0.00      0.00        24
                     _venements_de_vie       0.00      0.00      0.00        21
                    _pargne_disponible 